In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
import vpi
import time

fs = cv2.FileStorage("rectify_map_imx219_160deg_1080p.yaml", cv2.FILE_STORAGE_READ)

map_l = (
    fs.getNode('map_l_x').mat(),
    fs.getNode('map_l_y').mat()
)

map_r = (
    fs.getNode('map_r_x').mat(),
    fs.getNode('map_r_y').mat()
)

fs.release()

In [2]:
src_map_l = map_l[0]
idk_what_that_is_l = map_l[1]

map_l_y, map_l_x = map_l[0][:,:,0], map_l[0][:,:,1]
map_r_y, map_r_x = map_r[0][:,:,0], map_r[0][:,:,1]
map_r_x.shape

(1080, 1920)

In [3]:
# Reference map (identity mapping)
map_ref_x, map_ref_y = np.mgrid[0:1080:1, 0:1920:1]

## VPI remap

In [6]:
vpi_img = vpi.asimage(np.asarray(Image.open('image.jpg')))

In [7]:
warp = vpi.WarpMap(vpi.WarpGrid((1920,1080)))
wx,wy = np.asarray(warp).transpose(2,1,0)

x = wx-vpi_img.width/2
y = wy-vpi_img.height/2
 
R = vpi_img.height/4 # planet radius
r = np.sqrt(x*x + y*y)
 
theta = np.pi + np.arctan2(y,x)
phi = np.pi/2 - 2*np.arctan2(r, 2*R)
 
wx[:] = np.fmod((theta + np.pi)/(2*np.pi)*(vpi_img.width-1), vpi_img.width-1)
wy[:] = (phi + np.pi/2) / np.pi*(vpi_img.height-1)

In [8]:
t1 = time.perf_counter()

with vpi.Backend.CUDA:
    for _ in range(200):
        output = vpi_img.remap(warp)
        a = output.cpu()
        
t2 = time.perf_counter()

In [11]:
print(f"{200/(t2 - t1)} FPS")

116.38088856439751 FPS
